In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')


In [34]:
df = pd.read_csv('bench_copy.csv')
df.head()

,Emp Id,Associate,Grade,Department,Primary Skill,Current_Skill,Bench Start Date,Days on MTP,MTP_Days_Categories,Status,Sub Status,Action_Status,Availability,Work City,Aligned Opp2,Ops Comments,Comments,Line_Manager,Days_on_Bench
0,104,Daniel Park,G5,Engineering,Java,Java,15-10-2025,25,Short-Term,Aligned Opportunity,Awaiting BM Confirmation,Awaiting Confirmation,Yes,NY,Project C,Senior developer,Strong in backend,Lisa Chen,50
1,107,George Wang,G3,Engineering,JavaScript,React,20-10-2025,18,Short-Term,Available,Checking Opp,Awaiting TREQ,Yes,Seattle,Project F,Frontend dev,UI specialist,Tom Davis,45
2,111,Kevin O'Brien,G6,Engineering,Java,Spring Boot,25-10-2025,75,Long-Term,Not Available,Deal Movement,In Progress,Yes,Toronto,Project J,Tech lead,Architect level,Ryan Martinez,40
3,115,Oscar Martinez,G3,IT Support,Linux,Ubuntu,30-10-2025,10,Short-Term,Reserved,Extension,To be Confirmed,Yes,Dallas,Project N,Sys admin,Shell scripting,Mark Taylor,35
4,118,Rachel Green,G5,Engineering,Go,Kubernetes,02-11-2025,65,Long-Term,Aligned Opportunity,Ideal Bench,To be Exited,Yes,Atlanta,Project Q,DevOps eng,CI/CD expert,Sarah Brown,32


In [35]:
features_enhanced = ['Grade', 'Department', 'Primary Skill', 'Current_Skill', 'Work City', 
                     'Action_Status', 'Availability', 'Status', 'Sub Status', 'Bench Start Date']
target = 'Days_on_Bench'

data_enhanced = df[features_enhanced + [target]].copy()
print(data_enhanced.columns)

# Convert Bench Start Date to datetime and then to numerical value (days since reference date)
data_enhanced['Bench Start Date'] = pd.to_datetime(data_enhanced['Bench Start Date'], format='%d-%m-%Y')
reference_date = data_enhanced['Bench Start Date'].min()
data_enhanced['Bench_Start_Days'] = (data_enhanced['Bench Start Date'] - reference_date).dt.days

# 1. seperate the numerical value from the Grade like G5 --> 5, G10 --> 10
data_enhanced['Grade'] = data_enhanced['Grade'].str.extract('(\d+)').astype(int)

# 2. if both primary and current skill match its a skill match so the value will be 1 else 0
data_enhanced['Skill_Match'] = (data_enhanced['Primary Skill'] == data_enhanced['Current_Skill']).astype(int)

# 3. if the action status is approved then 1 else 0
data_enhanced['Is_Approved'] = (df['Action_Status'] == 'Approved').astype(int)

# 4. if availability is yes then 1 else 0
data_enhanced['Is_Available'] = (df['Availability'] == 'Yes').astype(int)

# 5. if action status is escalated then 1 else 0
data_enhanced['Is_Escalated'] = (df['Action_Status'] == 'Escalated').astype(int)

# 6. if Grade is greater than or equal to 5 then 1 else 0
data_enhanced['Is_Senior'] = (data_enhanced['Grade'] >= 5).astype(int)

# 7. Ready_Score
data_enhanced['Ready_Score'] = (data_enhanced['Is_Approved'] + data_enhanced['Is_Available'] + data_enhanced['Skill_Match'])

# 8. Risk_Score
# here we are doing 1 - skill match score because in risk we do need the negative factors of the skill match
data_enhanced['Risk_Score'] = (data_enhanced['Is_Escalated'] + (1 - data_enhanced['Skill_Match']) +data_enhanced['Is_Senior'])

# Encode categorical columns to assign them numerical values 
encoders_enhanced = {}
categorical_cols = ['Department', 'Primary Skill', 'Current_Skill', 'Work City','Action_Status', 'Availability', 'Status', 'Sub Status']

for col in categorical_cols:
    encoders_enhanced[col] = LabelEncoder()
    data_enhanced[col] = encoders_enhanced[col].fit_transform(data_enhanced[col].astype(str))

# Updated features list with Bench_Start_Days
features_final = ['Grade', 'Department', 'Primary Skill', 'Current_Skill', 'Work City', 'Action_Status', 'Availability', 'Status', 'Sub Status',
                  'Bench_Start_Days', 'Skill_Match', 'Is_Approved', 'Is_Available', 'Is_Escalated', 'Is_Senior', 'Ready_Score', 'Risk_Score']

# Store reference date for prediction function
bench_reference_date = reference_date

data_enhanced.head()

Index(['Grade', 'Department', 'Primary Skill', 'Current_Skill', 'Work City',
       'Action_Status', 'Availability', 'Status', 'Sub Status',
       'Bench Start Date', 'Days_on_Bench'],
      dtype='object')


,Grade,Department,Primary Skill,Current_Skill,Work City,Action_Status,Availability,Status,Sub Status,Bench Start Date,Days_on_Bench,Bench_Start_Days,Skill_Match,Is_Approved,Is_Available,Is_Escalated,Is_Senior,Ready_Score,Risk_Score
0,5,1,35,40,59,0,0,0,0,2025-10-15,50,0,1,0,1,0,1,2,1
1,3,1,36,76,87,1,0,1,1,2025-10-20,45,5,0,0,1,0,0,1,1
2,6,1,35,87,95,2,0,2,2,2025-10-25,40,10,0,0,1,0,1,1,2
3,3,2,41,94,21,3,0,3,3,2025-10-30,35,15,0,0,1,0,0,1,1
4,5,1,28,43,3,4,0,0,4,2025-11-02,32,18,0,0,1,0,1,1,2


In [36]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score

# asigning the features and target for training and testing the model
X_enhanced = data_enhanced[features_final]
y_enhanced = data_enhanced[target]

# segreting the data into training and testing sets in 70% and 30% ratio respectively keeping random state as 0 for reproducibility so that same split can be obtained every time the code is run
X_train_enh, X_test_enh, y_train_enh, y_test_enh = train_test_split(X_enhanced, y_enhanced, test_size=0.3, random_state=0)

model_enhanced = GradientBoostingRegressor(
    # Number of boosting trees  to fit
    n_estimators=100,           
    # Maximum depth of each tree 
    max_depth=3,       
    # Shrinks contribution of each tree         
    learning_rate=0.1,          
    # Minimum samples required to split an internal node 
    min_samples_split=10,      
    # Minimum samples required at each leaf node 
    min_samples_leaf=5,     
    # Fraction of samples used for fitting each tree     
    subsample=0.8,   
    # Fraction of features considered for best split           
    max_features=0.7, 
    # Fraction of training data for early stopping validation          
    validation_fraction=0.15,  
    # Stop training if no improvement for 15 iterations 
    n_iter_no_change=15,  
    # Seed for reproducibility      
    random_state=0              
)


model_enhanced.fit(X_train_enh, y_train_enh)

# as we are using negetive mean absolute error so we need to take negative of the cross val score to get the actual MAE value
cv_mae_enh = -cross_val_score(model_enhanced, X_train_enh, y_train_enh, cv=5, scoring='neg_mean_absolute_error').mean()

# this is compairing the actual bench data to the predicted data to give us the data how good it is fitting to the model
train_mae_enh = mean_absolute_error(y_train_enh, model_enhanced.predict(X_train_enh))
test_mae_enh = mean_absolute_error(y_test_enh, model_enhanced.predict(X_test_enh))
test_r2_enh = r2_score(y_test_enh, model_enhanced.predict(X_test_enh))


# Training MAE: How well the model fits the data it learned from (lower = better)
# CV MAE: Average error across 5 different validation splits - checks if the model is stable and not overfitting
# Test MAE & R²: Final performance on unseen data - the real measure of how accurate predictions will be for new employees

print(f"Training MAE: {train_mae_enh:.1f} days")
print(f"CV MAE: {cv_mae_enh:.1f} days")
print(f"Test MAE: {test_mae_enh:.1f} days : R²: {test_r2_enh:.3f}")

importance = pd.DataFrame({'Feature': features_final, 'Importance': model_enhanced.feature_importances_})
importance = importance.sort_values('Importance', ascending=False)
print("\nTop Features:")
for _, row in importance.head(5).iterrows():
    print(f"  {row['Feature']}: {row['Importance']:.3f}")

y_test_pred_enh = model_enhanced.predict(X_test_enh)
results_enh = list(zip(y_test_enh.values, y_test_pred_enh))
within_10 = sum(1 for a, p in results_enh if abs(a-p) <= 10)
print(f"\nWithin 10 days: {within_10}/{len(results_enh)}")

Training MAE: 0.5 days
CV MAE: 1.6 days
Test MAE: 1.2 days : R²: 0.984

Top Features:
  Bench_Start_Days: 0.844
  Primary Skill: 0.059
  Sub Status: 0.043
  Grade: 0.026
  Current_Skill: 0.014

Within 10 days: 33/33


In [37]:
def safe_encode(encoder, value):
    try:
        return encoder.transform([value])[0]
    except:
        return 0


def predict_bench_days_enhanced(grade, department, skill, current_skill, city, bench_start_date, action_status="Pending", availability="Yes", status="Active", sub_status="Available"):
    """
    Predict bench days for an employee.
    
    Parameters:
    - bench_start_date: Date string in format 'DD-MM-YYYY' (e.g., '15-10-2025')
    """
    
    grade_num = int(grade.replace('G', ''))
    
    # Convert bench start date to days since reference date
    bench_date = pd.to_datetime(bench_start_date, format='%d-%m-%Y')
    bench_start_days = (bench_date - bench_reference_date).days
    
    dept_enc = safe_encode(encoders_enhanced['Department'], department)
    skill_enc = safe_encode(encoders_enhanced['Primary Skill'], skill)
    curr_skill_enc = safe_encode(encoders_enhanced['Current_Skill'], current_skill)
    city_enc = safe_encode(encoders_enhanced['Work City'], city)
    action_enc = safe_encode(encoders_enhanced['Action_Status'], action_status)
    avail_enc = safe_encode(encoders_enhanced['Availability'], availability)
    status_enc = safe_encode(encoders_enhanced['Status'], status)
    sub_status_enc = safe_encode(encoders_enhanced['Sub Status'], sub_status)
    
    skill_match = 1 if skill == current_skill else 0
    is_approved = 1 if action_status == "Approved" else 0
    is_available = 1 if availability == "Yes" else 0
    is_escalated = 1 if action_status == "Escalated" else 0
    is_senior = 1 if grade_num >= 5 else 0
    ready_score = is_approved + is_available + skill_match
    risk_score = is_escalated + (1 - skill_match) + is_senior
    
    input_data = np.array([[
        grade_num, dept_enc, skill_enc, curr_skill_enc, city_enc,
        action_enc, avail_enc, status_enc, sub_status_enc,
        bench_start_days, skill_match, is_approved, is_available, is_escalated,
        is_senior, ready_score, risk_score
    ]])
    
    prediction = max(0, model_enhanced.predict(input_data)[0])
    
    if prediction <= 14:
        category = "Short-Term (Quick placement expected)"
    elif prediction <= 45:
        category = "Medium-Term (May need some time)"
    else:
        category = "Long-Term (Needs attention)"
    
    return prediction, category

In [38]:
grade = "G5"                    
department = "Data Science"      
skill = "Scikit-learn"         
current_skill = "XGBoost"          
city = "Toledo"
bench_start_date = "19-10-2025"  
action_status = "Pending"
availability = "Yes"
status = "Active"
sub_status = "Available"

days, category = predict_bench_days_enhanced(
    grade, department, skill, current_skill, city, bench_start_date,
    action_status, availability, status, sub_status
)

print(f"Employee: {grade} | {department}")
print(f"Skills: {skill} -> {current_skill}")
print(f"City: {city}")
print(f"Bench Start Date: {bench_start_date}")
print(f"Status: {action_status} | Available: {availability}")
print(f"Predicted Bench Days: {days:.1f} days")
print(f"Category: {category}")

Employee: G5 | Data Science
Skills: Scikit-learn -> XGBoost
City: Toledo
Bench Start Date: 19-10-2025
Status: Pending | Available: Yes
Predicted Bench Days: 46.4 days
Category: Long-Term (Needs attention)


In [39]:
import json

# Load the test data from JSON file
with open('bench_test_data.json', 'r') as f:
    test_employees = json.load(f)

print(f"{'Emp ID':<8} {'Actual':<10} {'Predicted':<12} {'Difference':<12} {'Match Status'}")
print("=" * 60)

matches = 0
total = len(test_employees)

for emp in test_employees:
    # Extract parameters from JSON record
    grade = emp['Grade']
    department = emp['Department']
    skill = emp['Primary Skill']
    current_skill = emp['Current_Skill']
    city = emp['Work City']
    bench_start_date = emp['Bench Start Date']
    action_status = emp['Action_Status']
    availability = emp['Availability']
    status = emp['Status']
    sub_status = emp['Sub Status']
    actual_days = emp['Days_on_Bench']
    
    # Predict using the trained model
    predicted_days, category = predict_bench_days_enhanced(
        grade, department, skill, current_skill, city, bench_start_date,
        action_status, availability, status, sub_status
    )
    
    # Calculate difference and determine if it's a match (within 10 days tolerance)
    difference = abs(actual_days - predicted_days)
    is_match = "Match" if difference <= 10 else " No Match"
    
    if difference <= 10:
        matches += 1
    
    print(f"{emp['Emp Id']:<8} {actual_days:<10} {predicted_days:<12.1f} {difference:<12.1f} {is_match}")

print("=" * 60)

print(f"\nSummary: {matches}/{total} predictions within 10 days tolerance ({matches/total*100:.1f}% accuracy)")


Emp ID   Actual     Predicted    Difference   Match Status
301      7          10.9         3.9          Match
302      47         48.0         1.0          Match
303      18         18.8         0.8          Match
304      50         46.9         3.1          Match
305      40         38.7         1.3          Match
306      49         46.0         3.0          Match
307      45         43.4         1.6          Match
308      11         12.0         1.0          Match
309      19         20.9         1.9          Match
310      31         32.4         1.4          Match
311      28         28.2         0.2          Match
312      28         27.4         0.6          Match
313      19         17.6         1.4          Match
314      33         35.3         2.3          Match
315      32         33.6         1.6          Match
316      39         37.7         1.3          Match
317      39         39.1         0.1          Match
318      33         33.4         0.4          Match
319  

In [40]:
def predict_from_json(json_file_path):
    """
    Read employee data from a JSON file and predict bench days for each employee.
    
    Parameters:
    - json_file_path: Path to the JSON file containing employee records
    
    Returns:
    - results_df: DataFrame with actual, predicted values and match status
    """
    import json
    
    # Load the test data from JSON file
    with open(json_file_path, 'r') as f:
        test_employees = json.load(f)
    
    print(f"Testing with: {json_file_path}")
    print(f"{'='*70}")
    print(f"{'Emp ID':<8} {'Name':<20} {'Actual':<10} {'Predicted':<12} {'Diff':<8} {'Status'}")
    print(f"{'='*70}")
    
    matches = 0
    total = len(test_employees)
    results = []
    
    for emp in test_employees:
        # Extract parameters from JSON record
        grade = emp['Grade']
        department = emp['Department']
        skill = emp['Primary Skill']
        current_skill = emp['Current_Skill']
        city = emp['Work City']
        bench_start_date = emp['Bench Start Date']
        action_status = emp['Action_Status']
        availability = emp['Availability']
        status = emp['Status']
        sub_status = emp['Sub Status']
        actual_days = emp['Days_on_Bench']
        
        # Predict using the trained model
        predicted_days, category = predict_bench_days_enhanced(
            grade, department, skill, current_skill, city, bench_start_date,
            action_status, availability, status, sub_status
        )
        
        # Calculate difference and determine if it's a match (within 10 days tolerance)
        difference = abs(actual_days - predicted_days)
        is_match = "✓ Match" if difference <= 10 else "✗ No Match"
        
        if difference <= 10:
            matches += 1
        
        print(f"{emp['Emp Id']:<8} {emp['Associate']:<20} {actual_days:<10} {round(predicted_days):<12} {round(difference):<8} {is_match}")
        
        results.append({
            'Emp Id': emp['Emp Id'],
            'Associate': emp['Associate'],
            'Actual': actual_days,
            'Predicted': round(predicted_days, 1),
            'Difference': round(difference, 1),
            'Match': difference <= 10
        })
    
    print(f"{'='*70}")
    accuracy = matches/total*100
    print(f"\n📊 Summary: {matches}/{total} predictions within 10 days tolerance ({accuracy:.1f}% accuracy)")
    
    return pd.DataFrame(results)

results_df = predict_from_json('bench_test_data.json')

Testing with: bench_test_data.json
Emp ID   Name                 Actual     Predicted    Diff     Status
301      Aria Stone           7          11           4        ✓ Match
302      Blake Hudson         47         48           1        ✓ Match
303      Chloe Bennett        18         19           1        ✓ Match
304      Drake Morrison       50         47           3        ✓ Match
305      Eva Mitchell         40         39           1        ✓ Match
306      Felix Grant          49         46           3        ✓ Match
307      Gina Torres          45         43           2        ✓ Match
308      Hugo Black           11         12           1        ✓ Match
309      Isla Reynolds        19         21           2        ✓ Match
310      Jake Sullivan        31         32           1        ✓ Match
311      Kira Patel           28         28           0        ✓ Match
312      Leo Chang            28         27           1        ✓ Match
313      Maya Foster          19         18